## Preprocessing

In [32]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from keras.callbacks import Callback
from tensorflow.keras.callbacks import ModelCheckpoint

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"], 1)

In [8]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df['APPLICATION_TYPE'].value_counts()
application_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_counts[application_counts < 420].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [12]:
# Look at CLASSIFICATION value counts for binning
class_counts = application_df['CLASSIFICATION'].value_counts()
class_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1580        1
C4120        1
C1236        1
C1570        1
C1732        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [15]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_great_one = class_counts[class_counts > 1]
class_count_great_one

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C2710        3
C0           3
C1260        3
C1256        2
C3200        2
C1267        2
C1246        2
C1234        2
Name: CLASSIFICATION, dtype: int64

In [17]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_counts[class_counts < 1500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_dummies = pd.get_dummies(application_df)
application_df_dummies.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [19]:
# Split our preprocessed data into our features and target arrays
X = application_df_dummies.drop(['IS_SUCCESSFUL'], axis = 'columns').values
y = application_df_dummies['IS_SUCCESSFUL'].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [40]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_layer = len(X_train[0])
hidden_node_layers1 = 90
hidden_node_layers2 = 45
hidden_node_layers3 = 1

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers1, input_dim = num_input_layer, activation = 'relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers2, activation = 'sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units = hidden_node_layers3, activation = 'sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 90)                3960      
                                                                 
 dense_4 (Dense)             (None, 45)                4095      
                                                                 
 dense_5 (Dense)             (None, 1)                 46        
                                                                 
Total params: 8,101
Trainable params: 8,101
Non-trainable params: 0
_________________________________________________________________


In [41]:
# Checkpoint file creation
os.makedirs('checkpoints/', exist_ok = True)
checkpoint_dir = r'checkpoints/Checkpoints.{epoch:02d}.hdf5'

In [42]:
# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

checkpoint = ModelCheckpoint(filepath = checkpoint_dir, monitor = 'accuracy', verbose = 1, save_weights_only = True,
                            mode = 'auto', period = 1)

In [43]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs = 150, callbacks = [checkpoint])

Epoch 1/150
747/804 [==========================>...] - ETA: 0s - loss: 0.5733 - accuracy: 0.7163
Epoch 1: saving model to checkpoints\Checkpoints.01.hdf5
804/804 [==============================] - 1s 750us/step - loss: 0.5729 - accuracy: 0.7172
Epoch 2/150
796/804 [============================>.] - ETA: 0s - loss: 0.5563 - accuracy: 0.7292
Epoch 2: saving model to checkpoints\Checkpoints.02.hdf5
804/804 [==============================] - 1s 709us/step - loss: 0.5563 - accuracy: 0.7290
Epoch 3/150
740/804 [==========================>...] - ETA: 0s - loss: 0.5531 - accuracy: 0.7305
Epoch 3: saving model to checkpoints\Checkpoints.03.hdf5
804/804 [==============================] - 1s 694us/step - loss: 0.5540 - accuracy: 0.7294
Epoch 4/150
790/804 [============================>.] - ETA: 0s - loss: 0.5520 - accuracy: 0.7325
Epoch 4: saving model to checkpoints\Checkpoints.04.hdf5
804/804 [==============================] - 1s 711us/step - loss: 0.5523 - accuracy: 0.7324
Epoch 5/150
742/804 

776/804 [===========================>..] - ETA: 0s - loss: 0.5409 - accuracy: 0.7340
Epoch 34: saving model to checkpoints\Checkpoints.34.hdf5
804/804 [==============================] - 1s 724us/step - loss: 0.5398 - accuracy: 0.7352
Epoch 35/150
744/804 [==========================>...] - ETA: 0s - loss: 0.5402 - accuracy: 0.7361
Epoch 35: saving model to checkpoints\Checkpoints.35.hdf5
804/804 [==============================] - 1s 690us/step - loss: 0.5400 - accuracy: 0.7365
Epoch 36/150
746/804 [==========================>...] - ETA: 0s - loss: 0.5397 - accuracy: 0.7369
Epoch 36: saving model to checkpoints\Checkpoints.36.hdf5
804/804 [==============================] - 1s 688us/step - loss: 0.5397 - accuracy: 0.7367
Epoch 37/150
744/804 [==========================>...] - ETA: 0s - loss: 0.5390 - accuracy: 0.7374
Epoch 37: saving model to checkpoints\Checkpoints.37.hdf5
804/804 [==============================] - 1s 686us/step - loss: 0.5398 - accuracy: 0.7357
Epoch 38/150
784/804 [===

747/804 [==========================>...] - ETA: 0s - loss: 0.5355 - accuracy: 0.7380
Epoch 67: saving model to checkpoints\Checkpoints.67.hdf5
804/804 [==============================] - 1s 684us/step - loss: 0.5350 - accuracy: 0.7383
Epoch 68/150
734/804 [==========================>...] - ETA: 0s - loss: 0.5358 - accuracy: 0.7378
Epoch 68: saving model to checkpoints\Checkpoints.68.hdf5
804/804 [==============================] - 1s 696us/step - loss: 0.5347 - accuracy: 0.7384
Epoch 69/150
741/804 [==========================>...] - ETA: 0s - loss: 0.5333 - accuracy: 0.7403
Epoch 69: saving model to checkpoints\Checkpoints.69.hdf5
804/804 [==============================] - 1s 691us/step - loss: 0.5345 - accuracy: 0.7395
Epoch 70/150
767/804 [===========================>..] - ETA: 0s - loss: 0.5353 - accuracy: 0.7382
Epoch 70: saving model to checkpoints\Checkpoints.70.hdf5
804/804 [==============================] - 1s 731us/step - loss: 0.5345 - accuracy: 0.7398
Epoch 71/150
794/804 [===

736/804 [==========================>...] - ETA: 0s - loss: 0.5334 - accuracy: 0.7386
Epoch 100: saving model to checkpoints\Checkpoints.100.hdf5
804/804 [==============================] - 1s 699us/step - loss: 0.5324 - accuracy: 0.7393
Epoch 101/150
735/804 [==========================>...] - ETA: 0s - loss: 0.5325 - accuracy: 0.7404
Epoch 101: saving model to checkpoints\Checkpoints.101.hdf5
804/804 [==============================] - 1s 696us/step - loss: 0.5327 - accuracy: 0.7396
Epoch 102/150
804/804 [==============================] - ETA: 0s - loss: 0.5328 - accuracy: 0.7400
Epoch 102: saving model to checkpoints\Checkpoints.102.hdf5
804/804 [==============================] - 1s 698us/step - loss: 0.5328 - accuracy: 0.7400
Epoch 103/150
777/804 [===========================>..] - ETA: 0s - loss: 0.5325 - accuracy: 0.7395
Epoch 103: saving model to checkpoints\Checkpoints.103.hdf5
804/804 [==============================] - 1s 724us/step - loss: 0.5327 - accuracy: 0.7394
Epoch 104/150


Epoch 133/150
736/804 [==========================>...] - ETA: 0s - loss: 0.5295 - accuracy: 0.7431
Epoch 133: saving model to checkpoints\Checkpoints.133.hdf5
804/804 [==============================] - 1s 691us/step - loss: 0.5313 - accuracy: 0.7410
Epoch 134/150
735/804 [==========================>...] - ETA: 0s - loss: 0.5306 - accuracy: 0.7413
Epoch 134: saving model to checkpoints\Checkpoints.134.hdf5
804/804 [==============================] - 1s 692us/step - loss: 0.5311 - accuracy: 0.7408
Epoch 135/150
791/804 [============================>.] - ETA: 0s - loss: 0.5314 - accuracy: 0.7412
Epoch 135: saving model to checkpoints\Checkpoints.135.hdf5
804/804 [==============================] - 1s 708us/step - loss: 0.5312 - accuracy: 0.7414
Epoch 136/150
741/804 [==========================>...] - ETA: 0s - loss: 0.5314 - accuracy: 0.7414
Epoch 136: saving model to checkpoints\Checkpoints.136.hdf5
804/804 [==============================] - 1s 690us/step - loss: 0.5314 - accuracy: 0.7405


In [44]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5678 - accuracy: 0.7292 - 227ms/epoch - 847us/step
Loss: 0.5678077340126038, Accuracy: 0.7292128205299377


In [45]:
# Export our model to HDF5 file
nn.save('deep_learning_optimization.h5')